In [1]:
import pandas as pd
import numpy as np
import re
import warnings

In [2]:
path = ''

In [3]:
df = pd.read_csv(path + '/data/data.csv', encoding = 'cp949')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2248 entries, 0 to 2247
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   판례일련번호  2248 non-null   int64 
 1   사건명     2248 non-null   object
 2   사건번호    2248 non-null   object
 3   선고일자    2248 non-null   object
 4   법원명     2248 non-null   object
 5   사건종류명   2248 non-null   object
 6   사건종류코드  2248 non-null   int64 
 7   판결유형    2248 non-null   object
 8   선고      2248 non-null   object
 9   판례상세링크  2248 non-null   object
 10  판시사항    2248 non-null   object
 11  판결요지    2248 non-null   object
 12  참조조문    2248 non-null   object
 13  참조판례    2246 non-null   object
 14  판례내용    2248 non-null   object
dtypes: int64(2), object(13)
memory usage: 263.6+ KB


# 1. 전처리

## 1.1. 한자 &rightarrow; 한글 변환

In [4]:
import hanja
from hanja import hangul

In [5]:
df['X'] = df['판례내용'].apply(lambda x : hanja.translate(x, 'substitution'))

## 1.2. 앞뒤 필요 없는 내용 제거

##### '유】' 또는 '유]'로 자르기

In [6]:
df['X'] = df['X'].apply(lambda x: ''.join(re.split(r'유】|유]', x)[1:]))

## 1.3. 숫자 전처리

##### 대법원 1992. 12. 22. 선고 92다14779 판결 &rightarrow; 제거

In [7]:
df['X'] = df['X'].apply(lambda x : re.sub('대법원 \d{4}\. \d{1,2}\. \d{1,2}\. 선고 \S+ 판결', '', x))

##### 날짜 : 2020. 7. 15. &rightarrow; 제거

In [8]:
df['X'] = df['X'].apply(lambda x : re.sub('\d{4}\. \d{1,2}\. \d{1,2}\.', '', x))

##### 조항 : 정규표현식으로 제거
제{숫자}조, 제{숫자}항, 제{숫자}호

In [9]:
df['X'] = df['X'].apply(lambda x : re.sub('제\\d+[조항호]', '', x))

##### 숫자 모두 제거(한글, 영어, 마침표만 남기기)

In [10]:
df['X'] = df['X'].apply(lambda x : re.sub('[^A-Za-z가-힣. ]', '', x))

##### 가. 나. 다. 와 같은 부분 제거

In [11]:
df['X'] = df['X'].apply(lambda x : re.sub('\W[가나다라마바사].', '', x))

In [12]:
# 파일 저장
df.to_csv(path + '/data/data_pre.csv', index = False, encoding = 'cp949')